In [2]:
days = 20*5

DemandA1 = [3000 3000 2000 2200 2200 rand(2000:3000)]

Dfactor = 60;
Efactor = 300;
A11factor = 15;

InvA1 = zeros(days+50,1)
InvD = zeros(days+50,1)
InvE = zeros(days+50,1)
InvA11 = zeros(days+50,1)

InvD[1:end] = 15;
InvE[1:end] = 60;
InvA11[1:end] = 30;

cycleA1 = 735

t=1;
demandMonth = 1
nOrders=0
noOrder = true
while t < days
    if t >= (demandMonth*20)
        InvA1[t:end] -= DemandA1[demandMonth]
        demandMonth += 1
    end
    if InvA1[t] > DemandA1[demandMonth]
        keepProduction = false
    else
        keepProduction = true
    end
    Dleft = (DemandA1[demandMonth] - InvA1[t]) * Dfactor
    Eleft = (DemandA1[demandMonth] - InvA1[t]) * Efactor
    A11left = (DemandA1[demandMonth] - InvA1[t]) * A11factor
    if ((InvD[t] < Dleft) || (InvE[t] < Eleft) || (InvA11[t] < A11left)) && noOrder && keepProduction
        nOrders+=1
        tLast = t
        tAdd = round(Int64,2*randn(1)[1] + 5)
        if tAdd < 0
            t+=0
        else
            t+=tAdd
        end
        
        InvD[tLast:t] = InvD[tLast]
        InvE[tLast:t] = InvE[tLast]
        InvA11[tLast:t] = InvA11[tLast]
        InvA1[tLast:t] = InvA1[tLast]
        
        InvD[t:end] += max(DemandA1[demandMonth]* Dfactor, cycleA1 * Dfactor * 7)
        InvE[t:end] += max(DemandA1[demandMonth]* Efactor, cycleA1 * Efactor * 7)
        InvA11[t:end] += max(DemandA1[demandMonth]* A11factor, cycleA1 * A11factor * 7)
    else
        if InvA1[t] > DemandA1[demandMonth]
            keepProduction = false
        else
            keepProduction = true
        end
        while ((InvD[t]-Dfactor)>0) && ((InvE[t]-Efactor)>0) && ((InvA11[t]-A11factor)>0) && ((t+1)<= days) && keepProduction          
            
            prod = round(Int64, 200*randn(1)[1] + cycleA1/2)
            nD = div(InvD[t], Dfactor)
            nE = div(InvE[t], Efactor)
            nA11 = div(InvA11[t], A11factor)
            
            nA1 = min(nD, nE, nA11, prod)
            
            InvA1[t:end] += nA1
            InvD[t:end] -= nA1*Dfactor
            InvE[t:end] -= nA1*Efactor
            InvA11[t:end] -= nA1*A11factor            
            defect = rand()/10
            
            InvA1[t:end] -= round(Int64, InvA1[t+1]*defect)
            if InvA1[t] > DemandA1[demandMonth]
                break
            end
            t+=1
            if t >= (demandMonth*20)
                InvA1[t:end] -= DemandA1[demandMonth]
                demandMonth += 1
            end

            Dleft = (DemandA1[demandMonth] - InvA1[t]) * Dfactor
            Eleft = (DemandA1[demandMonth] - InvA1[t]) * Efactor
            A11left = (DemandA1[demandMonth] - InvA1[t]) * A11factor
            if ((InvD[t] < Dleft) || (InvE[t] < Eleft) || (InvA11[t] < A11left)) && noOrder
                noOrder = false
                nOrders += 1
                tLast = t
                tAdd = round(Int64,2*randn(1)[1] + 5)
                if tAdd < 0
                    tOrder=0
                else
                    tOrder=tAdd+t
                end

                InvD[tLast:tOrder] = InvD[tLast]
                InvE[tLast:tOrder] = InvE[tLast]
                InvA11[tLast:tOrder] = InvA11[tLast]
                InvA1[tLast:tOrder] = InvA1[tLast]

            end
            if !noOrder
                if t == tOrder
                    noOrder = true
                    InvD[tOrder:end] += max(Dleft* Dfactor, cycleA1 * Dfactor * 7)
                    InvE[tOrder:end] += max(Eleft* Efactor, cycleA1 * Efactor * 7)
                    InvA11[tOrder:end] += max(A11left* A11factor, cycleA1 * A11factor * 7)
                end
            end
            
        end
    end
    t+=1
end




In [7]:
using Plots
plot(1:(days+50), InvD/Dfactor, label="D scaled", title="Inventory over 5 months")
plot!(1:(days+50), InvE/Efactor, label = "E scaled")
plot!(1:(days+50), InvA11/A11factor, label="A11 scaled")
plot!(1:(days+50), InvA1, label="A1 actual")
using Images
using FileIO
savefig("a1invall.png")


In [8]:
plot(1:(days+50), InvA1, label="A1", title="Inventory of A1")
using Images
using FileIO
savefig("a1inv.png")